In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wave
import random


/Users/tacky0612/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#種まき
random.seed(19950612)
np.random.seed(19950612)
tf.set_random_seed(19950612)

In [3]:
def wave_load(filename): #WAVの正規化とそれの吐き出し
#     open wave file
    wf = wave.open(filename,'r')
    channels = wf.getnchannels()

#     load wave data
    time = 1 #sec
    chunk_size =44100*time #1sec = 44100
    amp  = (2**8) ** wf.getsampwidth() / 2
    data = wf.readframes(chunk_size)   # バイナリ読み込み
    data = np.frombuffer(data,'int16') # intに変換
    data = data / amp                  # 振幅正規化(-1~1)
    data = data[::channels]        # stereo　→　monoral
    
    return data

In [4]:
def fft_load(filename,size): #FFT (窓あり)

    wave = wave_load(filename)
    
    st = 10000   # サンプリングする開始位置
    hammingWindow = np.hamming(size)    # ハミング窓
    fs = 44100 #サンプリングレート
    d = 1.0 / fs #サンプリングレートの逆数
    freqList = np.fft.fftfreq(size, d)
    
    windowedData = hammingWindow * wave[st:st+size]  # 切り出した波形データ（窓関数あり）
    data = np.fft.fft(windowedData)
    if max(abs(data)) == 0:    #無音がないか判定
        print("むりぽよ",filename)
    data = data / (max(abs(data))) # 0~1正規化

    return data

In [5]:
def data_set_train(count,instrument_list,start_list,end_list,size): #データセットを作りたい（願望）
    '''
    count = データセットとして取り出したい数 
    instrument_list = "楽器"(str)
    start_list =  始点
    end_list = 終点 
    size = FFTのサンプル数
    
    '''
    K = len(instrument_list)
    ffts = np.array([])
    labels = np.array([])
    
    for i in range(int(count)):
        
        N = random.randint(0,K-1)
        n = random.randint(start_list[N],end_list[N]) #ファイルの全範囲までの乱数生成
        filename = "./" + instrument_list[N] + "/output/" + str(n) +".wav"
        fft = fft_load(filename,size) # ランダムで指定楽器のFFTデータを配列に入れる
        ffts = np.append(ffts,fft,axis=0)
        label = np.zeros(K)                #1-of-Kのラベル生成
        label.put(N, 1)
        labels = np.append(labels,label,axis=0)
        num = len(fft)
        
    #np.ndarrayの整形
    ffts = np.reshape(ffts,(count,num))
    labels = np.reshape(labels,(count,K))
    
    return ffts,labels

In [6]:
def data_set_test(count,instrument_list,size): #データセットを作りたい（願望）
    '''
    count = データセットとして取り出したい数 
    instrument_list = "楽器"(str)
    size = FFTのサンプル数
    
    '''
    K = len(instrument_list)
    ffts = np.array([])
    labels = np.array([])
    
    i = 0
    for i in range(int(K)):
        n = 0
        for n in range(count):
            filename = "./" + instrument_list[i] + "/output/" + str(n) +".wav"
            fft = fft_load(filename,size) 
            ffts = np.append(ffts,fft,axis=0)
            label = np.zeros(K)                #1-of-Kのラベル生成
            label.put(i, 1)
            labels = np.append(labels,label,axis=0)
            
        
    #np.ndarrayの整形
    num = len(fft)
    ffts = np.reshape(ffts,(count*K,num))
    labels = np.reshape(labels,(count*K,K))
    
    return ffts,labels

In [7]:
namelist = ["synth","Aguitar"] #ここをイジる
startlist_train = [100] * len(namelist)
endlist_train = [1100,700] #データセットの終点を指定

In [8]:
num_units = 4096 #ノード
FFT = 1024 #FFTのサンプル数

x = tf.placeholder(tf.float64, [None, FFT])

w1 = tf.Variable(tf.truncated_normal([FFT, num_units],dtype=tf.float64))
b1 = tf.Variable(tf.zeros([num_units],dtype=tf.float64))
hidden1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, len(namelist)],dtype=tf.float64))
b0 = tf.Variable(tf.zeros([],dtype=tf.float64))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

t = tf.placeholder(tf.float64, [None,len(namelist)])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

i = 0
ffts_test, labels_test = data_set_test(100,namelist,FFT)
print(namelist)
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = data_set_train(200,namelist ,startlist_train,endlist_train,FFT)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:ffts_test, t: labels_test})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

['synth', 'Aguitar']


/Users/tacky0612/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Step: 100, Loss: 151.383131, Accuracy: 0.455000
Step: 200, Loss: 150.219969, Accuracy: 0.460000
Step: 300, Loss: 155.724429, Accuracy: 0.550000
Step: 400, Loss: 156.090173, Accuracy: 0.515000
Step: 500, Loss: 165.163079, Accuracy: 0.565000
Step: 600, Loss: 173.947078, Accuracy: 0.595000
Step: 700, Loss: 181.041481, Accuracy: 0.585000
Step: 800, Loss: 194.791092, Accuracy: 0.600000
Step: 900, Loss: 214.056358, Accuracy: 0.570000
Step: 1000, Loss: 238.582732, Accuracy: 0.530000
Step: 1100, Loss: 270.108545, Accuracy: 0.540000
Step: 1200, Loss: 271.080918, Accuracy: 0.570000
Step: 1300, Loss: 289.716689, Accuracy: 0.585000
Step: 1400, Loss: 309.723676, Accuracy: 0.565000
Step: 1500, Loss: 337.262528, Accuracy: 0.555000
Step: 1600, Loss: 362.260747, Accuracy: 0.525000
Step: 1700, Loss: 374.092045, Accuracy: 0.580000
Step: 1800, Loss: 405.855068, Accuracy: 0.500000
Step: 1900, Loss: 424.422863, Accuracy: 0.545000
Step: 2000, Loss: 448.102438, Accuracy: 0.550000
